#Use Case D - Using BERT 

*    This is a Google Colab notebook. You must have a Google account with a Google Drive to store/ load the model. Upload it from its location in GitHub and allow the code to access your Google Drive.
*    Fine-tuning can be skipped if already run once. One can go to cell "Loading the fine-tuned model", load the saved model and continue.
*    See instructions in cell "Select Bi-encoder model" to change the bi-encoder model.
*    See instructions in cell "Choose input dataset" to change the inputs - full set of proposals (datasets or SE Glossary articles).

##Installations and imports

In [ ]:
!pip install pyodbc

In [ ]:
!apt-get install virtuoso-opensource

Reading package lists... Done
Building dependency tree       
Reading state information... Done
virtuoso-opensource is already the newest version (6.1.6+repack-0ubuntu9).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [ ]:
!pip install sentence-transformers

In [ ]:
import pandas as pd
import numpy as np
import re
import pyodbc
import nltk, os
import sklearn
import ipywidgets as widgets
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import torch
import random
import ast

from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

from collections import defaultdict
from sentence_transformers import SentenceTransformer, SentencesDataset
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers.readers import LabelSentenceReader, InputExample
from torch.utils.data import DataLoader

from pathlib import Path
from google.colab import data_table, drive
from sentence_transformers import SentenceTransformer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

##Connect to the Virtuoso database

In [ ]:
c = pyodbc.connect('DRIVER=/usr/lib/odbc/virtodbc.so;HOST=lod.csd.auth.gr:1111;UID=kimon;PWD=RkhvQYZ442e2JVXLHdtW;DATABASE=ESTAT')

In [ ]:
#set encoding
c.setdecoding(pyodbc.SQL_CHAR, encoding='latin-1')
c.setencoding(encoding="latin-1")

In [ ]:
cursor = c.cursor()

##Select Bi-encoder model
To run with the non-default choice, interrupt execution, select a model and then "run after" from the next cell.

In [ ]:
# Load pre-trained model that we will use as a base for fine tuning
# By Default it loads the first option
# Warning: loading 2nd or 3rd model for finetuning requires large amounts of memory
bi_encoder_model_dropdown = widgets.Dropdown(options=["multi-qa-MiniLM-L6-cos-v1", "multi-qa-distilbert-cos-v1", "multi-qa-mpnet-base-cos-v1"])
display(widgets.HBox([widgets.Label(value="Bi-encoder model:"), bi_encoder_model_dropdown]))

In [ ]:
#We use the Bi-Encoder to encode all passages, so that we can use it with semantic search
print('Using model: ',bi_encoder_model_dropdown.value)
bi_encoder = SentenceTransformer(bi_encoder_model_dropdown.value)

Using model:  multi-qa-MiniLM-L6-cos-v1


## Load SE Glossary articles
*    Always loaded (even if fine-tuning is not run)
*    Definitions from dat_glossary.
*    Titles from dat_link_info (with resource_information_id=1, i.e. Eurostat, see ESTAT.V1.mod_resource_information).
*    Match above on id.

In [ ]:
# The SE Glossary articles are always loaded (even if fine tuning is not run)
query = 'SELECT T1.id, T2.title, T1.definition, T2.url FROM ESTAT.V1.dat_glossary AS T1 INNER JOIN ESTAT.V1.dat_link_info AS T2 ON T1.id=T2.id WHERE T2.resource_information_id=1'
cursor.execute(query)
t1 = cursor.fetchall()
GL_df = pd.DataFrame.from_records(t1, columns=[x[0] for x in cursor.description])
# also keep a copy of this query results
glossary = GL_df.copy()


##Fine tuning

In [ ]:
def clean(x, quotes=True):
    if pd.isnull(x): return x  
    x = x.strip()
    
    ## make letter-question mark-letter -> letter-quote-space-letter !!! but NOT in the lists of URLs!!!
    if quotes:
        x = re.sub(r'([A-Za-z])\?([A-Za-z])','\\1\' \\2',x) 
    
    ## make letter-question mark-space lower case letter letter-quote-space letter
    x = re.sub(r'([A-Za-z])\? ([a-z])','\\1\' \\2',x) 

    ## delete ,000 commas in numbers    
    x = re.sub(r'\b(\d+),(\d+)\b','\\1\\2',x) ## CORRECTED
    
    ## delete  000 spaces in numbers
    x = re.sub(r'\b(\d+) (\d+)\b','\\1\\2',x) ## CORRECTED
    
    ## remove more than one spaces
    x = re.sub(r' +', ' ',x)
    
    ## remove start and end spaces
    x = re.sub(r'^ +| +$', '',x,flags=re.MULTILINE) 
    
    ## space-comma -> comma
    x = re.sub(r' \,',',',x)
    
    ## space-dot -> dot
    x = re.sub(r' \.','.',x)
    
    x = re.sub(r'â.{2}',"'",x) ### !!! NEW: single quotes are read as: âXX
    
    return x

### Delete records with empty definitions and carry out data cleansing


In [ ]:
GL_df['definition'] = GL_df['definition'].replace('', np.nan)
GL_df.dropna(axis=0,subset=['definition'],inplace=True)

GL_df.reset_index(drop=True, inplace=True)

GL_df['title'] = GL_df['title'].apply(clean)
GL_df['title'] = GL_df['title'].apply(lambda x: re.sub(r'\?','-',x)) ## also replace question marks by dashes
GL_df['definition'] = GL_df['definition'].apply(clean)

### Delete "special" records

* i.e. redirections.

In [ ]:
# Drop The records with definitions "The revision ..." and "Redirect to ..." 

idx = GL_df[GL_df['definition'].str.startswith('The revision #')].index
GL_df.drop(idx , inplace=True)
idx = GL_df[GL_df['definition'].str.startswith('Redirect to')].index
GL_df.drop(idx , inplace=True)
# dropping some pairs that definition is something generic with a link 
idx = GL_df[GL_df['definition'].str.startswith('<Brief user-oriented definition')].index 
GL_df.drop(idx , inplace=True)

GL_df.reset_index(drop=True, inplace=True)

GL_df.rename(columns={'id':'article_id'},inplace=True)
GL_df.rename(columns={'definition':'content'},inplace=True)

### Add paragraphs titles and contents from the SE articles

* From dat_article_paragraph with abstract=0 (i.e. "no").
* Match article_id from dat_article_paragraph with id from dat_article.
* Carry out data cleansing on titles and paragraph contents.

In [ ]:
query = 'SELECT article_id, title, content FROM ESTAT.V1.dat_article_paragraph WHERE abstract=0 AND article_id IN (SELECT id FROM ESTAT.V1.dat_article)'
cursor.execute(query)
t2 = cursor.fetchall()
add_content = pd.DataFrame.from_records(t2, columns=[x[0] for x in cursor.description])

In [ ]:
add_content['title'] = add_content['title'].apply(clean)
add_content['content'] = add_content['content'].apply(clean)

add_content['title'] = add_content['title'].replace('', np.nan)
add_content = add_content.dropna(axis=0,subset=['title'])
add_content['content'] = add_content['content'].replace('', np.nan)
add_content = add_content.dropna(axis=0,subset=['content'])

### Query to get OECD Glossary data

In [ ]:
query = "SELECT id, term,definition,context,theme FROM ESTAT.V1.OECD_Glossary"
OECD_df = pd.read_sql(query,c)
OECD_df['definition'] = OECD_df['definition'].apply(clean)

In [ ]:
#apply cleaning
OECD_df['term'] = OECD_df['term'].apply(clean)
OECD_df['context'] = OECD_df['context'].apply(clean).apply(lambda x: re.sub(r'\. \. ','. ',x)).apply(lambda x: re.sub(r'\?','-',x)) ## also replace question marks by dashes
OECD_df['definition'] = OECD_df['definition'].apply(clean).apply(lambda x: re.sub(r'\. \. ','. ',x)).apply(lambda x: re.sub(r'\?','-',x)) ## also replace question marks by dashes

#renaming
OECD_df.rename(columns={'id':'article_id'},inplace=True)
OECD_df.rename(columns={'term':'title'},inplace=True)

# using title-definition pairs
OECD_df1 = OECD_df[['article_id','title','definition']]
OECD_df1 = OECD_df1.rename(columns={'definition':'content'})

# using title-context pairs
OECD_df2 = OECD_df[['article_id','title','context']]
OECD_df2 = OECD_df2.rename(columns={'context':'content'})

OECD_df1 = OECD_df1[['article_id','title','content']]
OECD_df2 = OECD_df2[['article_id','title','content']]

# deleting empty
OECD_df1['content'] = OECD_df1['content'].replace('', np.nan)
OECD_df1 = OECD_df1.dropna(axis=0,subset=['content'])
OECD_df1['title'] = OECD_df1['title'].replace('', np.nan)
OECD_df1 = OECD_df1.dropna(axis=0,subset=['title'])

OECD_df2['content'] = OECD_df2['content'].replace('', np.nan)
OECD_df2 = OECD_df2.dropna(axis=0,subset=['content'])
OECD_df2['title'] = OECD_df2['title'].replace('', np.nan)
OECD_df2 = OECD_df2.dropna(axis=0,subset=['title'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


###Abstracts and context sections from SE articles

In [ ]:
query = 'SELECT T1.id, T1.context, T2.title, T3.content FROM ESTAT.V1.dat_article as T1 INNER JOIN ESTAT.V1.dat_link_info as T2 ON T1.id=T2.id INNER JOIN ESTAT.V1.dat_article_paragraph as T3 ON T2.id=T3.article_id WHERE T2.resource_information_id=1 AND T3.abstract=1'
cursor.execute(query)
t2 = cursor.fetchall()
SE_df = pd.DataFrame.from_records(t2, columns=[x[0] for x in cursor.description])

In [ ]:
SE_df.rename(columns={'id':'article_id','content':'abstract'},inplace=True)

In [ ]:
# apply cleaning
SE_df['title'] = SE_df['title'].apply(clean).apply(lambda x: re.sub(r'\?','-',x)) ## also replace question marks by dashes
SE_df['context'] = SE_df['context'].apply(clean)
SE_df['abstract'] = SE_df['abstract'].apply(clean)

# using title-context pairs
SE_df1 = SE_df[['article_id','title','context']]
SE_df1 = SE_df1.rename(columns={'context':'content'})

# using title-abstract pairs
SE_df2 = SE_df[['article_id','title','abstract']]
SE_df2 = SE_df2.rename(columns={'abstract':'content'})

# deleting empty
SE_df1['content'] = SE_df1['content'].replace('', np.nan)
SE_df1 = SE_df1.dropna(axis=0,subset=['content'])

SE_df2['content'] = SE_df2['content'].replace('', np.nan).replace('nan', np.nan)
SE_df2 = SE_df2.dropna(axis=0,subset=['content'])

###Merging all available glossaries and articles

In [ ]:
# merge OECD glossary with add_content, GL_df and SE_df
glossaries = [OECD_df1, OECD_df2, GL_df, add_content, SE_df1, SE_df2]
merged_glossary = pd.concat(glossaries, ignore_index=True)
merged_glossary = merged_glossary.drop(['url'],axis=1)

del(OECD_df1, OECD_df2, GL_df, add_content, SE_df1, SE_df2)

In [ ]:
# setting all pairs of titles and content to have a label of 1
# this is because all pairs are considered a positive example
merged_glossary['is_pair']=1

### Keep 80% of data for training and 20% for evaluation of the model

In [ ]:
from sklearn.model_selection import train_test_split
train_features, eval_features, train_labels, eval_labels = train_test_split(merged_glossary[['title','content']], 
                                                                            merged_glossary[['is_pair']], 
                                                                            train_size=0.8, 
                                                                            random_state=1,
                                                                            stratify=merged_glossary['is_pair'])

train_features = train_features.reset_index(drop=True)
eval_features = eval_features.reset_index(drop=True)

###Fine tuning bi-encoder (CUDA required)

In [ ]:
# convert training examples to the appropriate format
train_examples = []
for i in range(len(train_features)):
  train_examples.append(InputExample(texts=[train_features['title'].iloc[i], train_features['content'].iloc[i]]))

In [ ]:
finetune_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

In [ ]:
# MultipleNegativesRankingLoss is a great loss function if you only have positive pairs 
loss = MultipleNegativesRankingLoss(model=bi_encoder)

In [ ]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, BinaryClassificationEvaluator, MSEEvaluator
from sentence_transformers import evaluation

sentences1 = list(eval_features['title'])
sentences2 = list(eval_features['content'])
scores_eval = list(eval_labels['is_pair'])

In [ ]:
evaluator = evaluation.MSEEvaluator(sentences1, sentences2, teacher_model=bi_encoder)

In [ ]:
#cleaning up any CUDA related left-overs
import gc
gc.collect()

80

In [ ]:
# Fine-tune the model (BE CAREFUL TO SET A PATH FOR THE FINE-TUNED MODEL)

num_epochs = 10
bi_encoder.fit(train_objectives=[(finetune_dataloader, loss)], evaluator=evaluator, epochs=num_epochs,output_path='/content/drive/MyDrive/testing_fine_tuning')


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/744 [00:00<?, ?it/s]

Iteration:   0%|          | 0/744 [00:00<?, ?it/s]

Iteration:   0%|          | 0/744 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/744 [00:00<?, ?it/s]

Iteration:   0%|          | 0/744 [00:00<?, ?it/s]

Iteration:   0%|          | 0/744 [00:00<?, ?it/s]

Iteration:   0%|          | 0/744 [00:00<?, ?it/s]

Iteration:   0%|          | 0/744 [00:00<?, ?it/s]

##Loading the fine-tuned model

Run from here to skip the fine-tuning if already run.

In [ ]:
fine_tuned_bi_encoder = SentenceTransformer('/content/drive/MyDrive/testing_fine_tuning')

In [ ]:
bi_encoder = fine_tuned_bi_encoder

##Loading the datasets information from the database

In [ ]:
query = 'SELECT id, number, codes, names, file_descr, file_code, level, link FROM ESTAT.V1.dat_all_datasets'
cursor.execute(query)
t1 = cursor.fetchall()
crumbs_df = pd.DataFrame.from_records(t1, columns=[x[0] for x in cursor.description])

crumbs_df['codes'] = crumbs_df['codes'].apply(lambda x: ast.literal_eval(x))
crumbs_df['names'] = crumbs_df['names'].apply(lambda x: ast.literal_eval(x))
crumbs_df.rename(columns={'number':'Number','codes':'Codes','names':'Names','file_descr':'File description',
                               'level':'Level','file_code':'File code','link':'Link'},inplace=True)

## keep only leaves - datasets
idx = crumbs_df[crumbs_df['File code']==''].index
crumbs_df.drop(index=idx, inplace=True) 
crumbs_df.reset_index(drop=True,inplace=True)

crumbs_df['Full path'] = crumbs_df['Names'].apply(lambda x: [y for y in x[1:]]) ## exclude first part
crumbs_df['Full path'] = crumbs_df['Full path'].apply(lambda x: ' '.join(x))

crumbs_df = crumbs_df[['Number','Codes','Names','File description','File code','Level','Full path','Link']]

In [ ]:
# Drop duplicates with the same file code
crumbs_df = crumbs_df.drop_duplicates(subset='File code', keep="first").reset_index()

## Loading the SE Glossary articles from the database
*    Already loaded as glossary

In [ ]:
## Skip if already run
if 'File description' not in glossary.columns:

    glossary['definition'] = glossary['definition'].replace('', np.nan)
    glossary = glossary.dropna(axis=0,subset=['definition'])

    glossary.reset_index(drop=True, inplace=True)

    glossary['title'] = glossary['title'].apply(clean)
    glossary['title'] = glossary['title'].apply(lambda x: re.sub(r'\?','-',x)) ## also replace question marks by dashes
    # Edit titles in order to remove "Glossary:"
    glossary['title'] = glossary['title'].apply(lambda x: re.sub(r'Glossary:',' ',x)) 
    glossary['definition'] = glossary['definition'].apply(clean)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [ ]:
## Skip if already run
if 'File description' not in glossary.columns:
    # Drop The records with definitions "The revision ..." and "Redirect to ..." 
    idx = glossary[glossary['definition'].str.startswith('The revision #')].index
    glossary.drop(idx , inplace=True)
    idx = glossary[glossary['definition'].str.startswith('Redirect to')].index
    glossary.drop(idx , inplace=True)
    # dropping some pairs that definition is something generic with a link 
    idx = glossary[glossary['definition'].str.startswith('<Brief user-oriented definition')].index 
    glossary.drop(idx , inplace=True)

    glossary.reset_index(drop=True, inplace=True)

In [ ]:
## Skip if already run
if 'File description' not in glossary.columns:
    glossary.rename(columns={'id':'article_id'},inplace=True)
    glossary.rename(columns={'url':'Link'},inplace=True)
    glossary.rename(columns={'definition':'File description'},inplace=True)

    glossary['Full path'] = glossary['title'] + '. ' + glossary['File description']

##Save datasets and SE Glossary articles data

In [ ]:
# save Eurostat's datasets to pickle 
crumbs_df.to_pickle('/content/drive/MyDrive/datasets_use_case_d.pkl')

In [ ]:
# save glossary articles to pickle 
glossary.to_pickle('/content/drive/MyDrive/glossary_use_case_d.pkl')

##Choose input dataset
To run with the non-default choice, interrupt execution, select the second choice and then "run after" from the next cell. 

In [ ]:
# By Default it loads the first option

input_dataset_dropdown = widgets.Dropdown(options=["datasets_use_case_d.pkl", "glossary_use_case_d.pkl"])
display(widgets.HBox([widgets.Label(value="Input dataset:"), input_dataset_dropdown]))



In [ ]:
#We use the Bi-Encoder to encode all passages, so that we can use it with sematic search
print('Using ',input_dataset_dropdown.value)
datasets_path = '/content/drive/MyDrive/' + input_dataset_dropdown.value
crumbs_df = pd.read_pickle(datasets_path)

Using  glossary_use_case_d.pkl


##Calculate corpus embeddings in advance

In [ ]:
# We encode all corpus into our vector space, using "Full path" column. 
corpus_embeddings_full = bi_encoder.encode(crumbs_df['Full path'], convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/40 [00:00<?, ?it/s]

In [ ]:
crumbs_df['embeddings'] = list(corpus_embeddings_full.cpu())

In [ ]:
#pre-download cross-encoder
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

## Functions

In [ ]:
# A function that returns the most relevant datasets (along with some other data)
def relevant_datasets(query, data_df, howmany, top_k=100):

    data_df = data_df[['File description', 'Full path', 'Link', 'embeddings']].reset_index(drop=True).copy()
    # Encode query into our vector space 
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    #top_k  refers to number of Eurostat databases we want to retrieve with the bi-encoder
    hits = util.semantic_search(question_embedding, torch.stack(data_df['embeddings'].values.tolist()), top_k=top_k)
    hits = hits[0]  # Get the hits for the first query
    hits_df = pd.DataFrame(hits)

    data_df['corpus_id'] = data_df.index
    data_df = hits_df.merge(data_df, on='corpus_id', how='inner')

    #Re-ranking
    #Loading a cross-encoder model, and re-ranking the results of the bi-encoder
    #Cross-Encoder achieve higher performance than Bi-Encoders, however, they do not
    #scale well for large datasets. Here, it can make sense to combine Cross- and
    #Bi-Encoders, for Semantic Search scenarios: 
    #First, you use an efficient Bi-Encoder to retrieve e.g. the top-100 most
    #similar sentences for a query. Then, you use a Cross-Encoder to re-rank
    #these 100 hits by computing the score for every (query, hit) combination.

    # Now, score all retrieved passages with the cross_encoder
    cross_inp = [[query, data_df['Full path'][hit]] for hit in range(len(data_df))]

    data_df['cross_scores'] = cross_encoder.predict(cross_inp)
    data_df = data_df.sort_values(['cross_scores'],ascending=False)
    results = data_df[['cross_scores', 'score','File description', 'Full path', 'Link', 'embeddings' ]].head(howmany)

    return results

In [ ]:
# Notion behind this function is that when user answers 'no' to a proposed ngram
# or replies with their own phrase, then they might be interested in more generic
# ngram (e.g. if asked 'educational attainment level' and answered 'no', might
# be interested in 'educational' solely). # Simultaneously, the user would not
# want to see a more specific ngram (e.g. if asked for 'unemployment rate', wouldn't
# be interested in 'unemployment rate by sex').
# This function is used when we want to find all similar terms to input_term,
# against the terms_list. We seek similar terms in terms_list.  If input term is
# a subset of a (stemmed) term in the terms_list, then this is considered as similar.
# Example:
# input_term = unemployment rate
# terms_list = [unemployment rate, unemployment rates, unemployment, rate, harmonised unemployment rate]
# indices_list = [0,1,4]
def indices_of_similar_subset_terms(terms_list, input_term):
  # define stop words language
  stop_words = stopwords.words('english')

  # input_term might be 1 or more tokens (usually words, numbers, etc)
  tokens = input_term.split()
  # get all unique input tokens that are not stopwords
  tokens = set(filter(lambda o: not o in stop_words, tokens))
  if len(tokens)==0:
    return []
  
  # get the unique stemmed tokens
  stemmer = PorterStemmer()
  stemmed_tokens = set(map(lambda o: stemmer.stem(o), tokens))

  # for every term in the term_list get all
  indices_list = []
  for i,term in enumerate(terms_list):
    stemmed_term = set(map(lambda o: stemmer.stem(o), term.split()))
    # issubset returns true if stemmed_tokens is a subset of stemmed_term
    if stemmed_tokens.issubset(stemmed_term):
      indices_list.append(i)
    
  return indices_list

# Notion behind this function is that when user answers 'yes' to a proposed ngram,
# then they might be interested in more specific ngram (e.g. if asked
# 'unemployment rate', would be interested in 'harmonised unemployment rate' also).
# Simultaneously, the user would not want to see a repetition of a less specific ngram
# (e.g. if asked for 'unemployment rate', wouldn't be interested in either
# 'unemployment' or 'rates' to be asked again).
# This function is used when we want to find all similar terms to input_term,
# against the terms_list. We seek similar terms in terms_list.  If input term is
# a superset of a (stemmed) term in the terms_list, then this is considered as similar.
# Example:
# input_term = unemployment rate
# terms_list = [unemployment rate, unemployment rates, unemployment, rate, harmonised unemployment rate]
# indices_list = [0, 1, 2, 3]
def indices_of_similar_superset_terms(terms_list, input_term):
  stop_words = stopwords.words('english')

  tokens = input_term.split() 
  tokens = set(filter(lambda o: not o in stop_words, tokens))
  if len(tokens)==0:
    return []
  stemmer = PorterStemmer()
  stemmed_tokens = set(map(lambda o: stemmer.stem(o), tokens))

  indices_list = []
  for i,term in enumerate(terms_list):
    stemmed_term = set(map(lambda o: stemmer.stem(o), term.split()))
    stemmed_term = set(filter(lambda o: not o in stop_words, stemmed_term))
    if stemmed_tokens.issuperset(stemmed_term):
      indices_list.append(i)
    
  return indices_list

# remove 'by' if it is the last token
def clean_term_to_show(input_term):
  tokens = input_term.split()
  if tokens[-1]=='by':
    return ' '.join(tokens[:-1])
  return input_term

# function to produce a random question 
def choose_question(input_term):
  
  input_tokens = input_term.split()
  # if ngram starts with 'by'
  if input_tokens[0]=='by':
    question_texts = [f"Would you like to see results {input_term}?", 
                      f"Would you be interested in seeing results {input_term}?",  
                      f"Let's make it more specific. Are you interested in seeing results {input_term}?"]
  # if ngram contains 3 or more tokens (this means that the question becomes very specific)  
  elif len(input_tokens)>=3:
    question_texts = [f"What about {input_term}?"]
  else:
    question_texts = [f"Can we make it more specific? Is '{input_term}' related to what you are looking for?", 
                      f"Let's make it more specific. Are you interested in '{input_term}'?",
                      f"I have some results related to '{input_term}'. Is that what you are looking for?",
                      f"It would help me if could be could be more specific. Is '{input_term}' related to what you are searching?",
                      f"Would you be interested in '{input_term}'?",
                      f"Is '{input_term}' related to what you are looking for?",
                      f"It would help me if you could provide more details. Is '{input_term}' something related to your query?",
                      f"Please provide more details. Is '{input_term}' something related to what you are looking for?",
                      f"Are you interested to results related to '{input_term}'?",
                      f"Help me please to understand what you mean. Are you referring to '{input_term}'"
    ]
  return random.choice(question_texts)

# generate a list tha contains all 1 to 4 ngrams from input_text
def generate_ngrams(input_text):
  vectorizer1 = TfidfVectorizer(ngram_range=(1, 4))
  _ = vectorizer1.fit([input_text])
  input_ngrams = list(vectorizer1.get_feature_names_out())
  return input_ngrams

# function to create histograms of 2 clusters (for K-mean results)
def create_plot(datasets):
  # use a gray background
  ax = plt.axes(facecolor='#E6E6E6')
  ax.set_axisbelow(True)

  # draw solid white grid lines
  plt.grid(color='w', linestyle='solid')

  # hide axis spines
  for spine in ax.spines.values():
      spine.set_visible(False)
      
  # hide top and right ticks
  ax.xaxis.tick_bottom()
  ax.yaxis.tick_left()

  # lighten ticks and labels
  ax.tick_params(colors='gray', direction='out')
  for tick in ax.get_xticklabels():
      tick.set_color('gray')
  for tick in ax.get_yticklabels():
      tick.set_color('gray')

  ax.hist(datasets['cross_scores'][(datasets['cluster_id'] == 0)], 5, density=False, histtype='bar',  label='cluster 1')
  ax.hist(datasets['cross_scores'][(datasets['cluster_id'] == 1)], 5, density=False, histtype='bar',  label='cluster 2')
  ax.legend(prop={'size': 10})
  ax.set_xlabel('cross_scores')
  ax.set_title('Reduction phase, K-means clusters histogram')
  plt.pause(0.05)
  return ax

##Chatbot

In [ ]:
stop_words = stopwords.words('english')
# exclude 'by' from stopwords (as we think that carries useful information)
stop_words.remove('by')

In [ ]:
def chatbot(crumbs_df, stopwords, howmany_v, stop_after_multiple_no, stop_after_multiple_yes):
  no_words = []
  input_text = input("bot: Hello, what are you looking for? \nme:  ")
  query_results_df = relevant_datasets(input_text, data_df=crumbs_df, howmany=howmany_v)

  # keep asking the user for clarifications up until the results<=5, or if the user replied a total of 6 'no'
  while stop_after_multiple_yes>0 and stop_after_multiple_no>0:

    # perform kmeans with 2 clusters on cross_scores, and only keep the cluster that contains the result with best cross-score
    kmeans = KMeans(n_clusters=2, random_state=42).fit(query_results_df[['cross_scores']])
    query_results_df['cluster_id'] =  list(kmeans.labels_)
    # display clustering results
    
    ### COMMENT-OUT TO SKIP THIS DISPLAY 
    ax = create_plot(query_results_df)
    display(ax)
    
    # reduce proposals
    query_results_df = query_results_df[query_results_df['cluster_id']==(query_results_df['cluster_id'].iloc[0])]

    if len(query_results_df)<=5:
      break

    # display intermediate results
    ## COMMENT-OUT TO SKIP THIS DISPLAY 
    display(data_table.DataTable(query_results_df.drop(['embeddings'], axis=1), num_rows_per_page=10))

    vectorizer = TfidfVectorizer(stop_words=stop_words, ngram_range=(2, 4), max_df=0.89, min_df=0.02, token_pattern='(?u)\\b\\w*[a-zA-Z#$%&-/<=>]\\w*\\b')
    subset = vectorizer.fit_transform(query_results_df['File description'])
    # dictionary that containes all possible 2-4 ngrams that have been produced by datasets descriptions
    words_dict = vectorizer.vocabulary_

    # get all 1-4 ngrams from the user input
    input_text2words = generate_ngrams(input_text)
    # list of all ngrams that we don't want to propose (either the ones that user said no, or anything related to their query)
    all_unwanted_words = input_text2words + no_words

    # a vocublulary of tokens that includes all ngrams that have been produced 
    vocabulary_words = vectorizer.get_feature_names_out()

    # decide on the order that we want to show the user the ngrams (in the clarification question)
    sorted_word_idx = list(np.argsort(np.mean(subset.toarray(), axis=0)))

    # get the index of all unwanted words that are in the words dictionary
    existing_idx = list(map(lambda o: words_dict[o], filter(lambda o: o in words_dict, all_unwanted_words)))

    while stop_after_multiple_no>0:

      # get the best ngram, according to tf-idf, that is not in unwanted words
      question_word_idx = [x for x in sorted_word_idx if x not in existing_idx][-1]
      question_word = vocabulary_words[question_word_idx]

      cleaned_term = clean_term_to_show(question_word)
      # making sure that the cleaned term will be removed from ngrams to propose, and added to unwanted words 
      if cleaned_term in all_unwanted_words:
        no_words.append(cleaned_term)
        sorted_word_idx.remove(question_word_idx)
        continue

      answer_text = input(f"bot: {choose_question(cleaned_term)} (yes/no or add some keywords) \nme:  ") 

      if answer_text.lower()=='yes':
        # stop if you reach a threshold for "yes" answers from the user
        stop_after_multiple_yes = stop_after_multiple_yes - 1

        #update the initial query with the new ngram that we proposed (question_word)
        input_text = input_text + ' ' + question_word
        # find all terms that are similar to that ngram proposed
        similar_terms_idx = indices_of_similar_superset_terms(vocabulary_words, question_word)
        # add all similar terms to the unwanted words
        no_words.extend(list(map(lambda o: vocabulary_words[o], similar_terms_idx)))

        for similar_term_idx in similar_terms_idx:
          # remove all similar terms from the vocabulary that we use to propose new ngrams
          if similar_term_idx in sorted_word_idx: 
            sorted_word_idx.remove(similar_term_idx)
        break

      elif answer_text.lower()=='no':
        # stop if you reach a threshold for "no" answers from the user
        stop_after_multiple_no = stop_after_multiple_no - 1

        # find all terms that are similar to that ngram proposed  
        similar_terms_idx = indices_of_similar_subset_terms(vocabulary_words, question_word)
        # add all similar terms to the unwanted words
        no_words.extend(list(map(lambda o: vocabulary_words[o], similar_terms_idx)))

        # remove all similar terms from the vocabulary that we use to propose new ngrams
        for similar_term_idx in similar_terms_idx:
          if similar_term_idx in sorted_word_idx:
            sorted_word_idx.remove(similar_term_idx)

      # user can quit at any time
      elif answer_text.lower()=='q':
        stop_after_multiple_no = 0
        break

      # in case that the user has entered a keyword/phrase/sentence instead of answering yes/no  
      else:
        stop_after_multiple_yes = stop_after_multiple_yes - 1

        #update the initial query with the new ngram that we proposed (question_word)
        input_text = input_text + ' ' + answer_text
        # get all ngrams from the user's answer 
        answer_text2words = generate_ngrams(answer_text)
        # for every token that the user's answer contain
        for token in answer_text2words:
          # find all terms that are similar to the user's answer  
          similar_terms_idx = indices_of_similar_superset_terms(vocabulary_words, token)
          # add all similar terms to the unwanted words
          no_words.extend(list(map(lambda o: vocabulary_words[o], similar_terms_idx)))

          # remove all similar terms from the vocabulary that we use to propose new ngrams
          for similar_term_idx in similar_terms_idx:
            if similar_term_idx in sorted_word_idx:
              sorted_word_idx.remove(similar_term_idx)
        break
    
    # get the updated results based on the augmented user's answer
    query_results_df = relevant_datasets(input_text, data_df=query_results_df, howmany=howmany_v)

  # final results (truncated so as to show only top5 to the user)
  query_results_df = query_results_df.head(5)
  print('bot: Thank you. Here are the most relevant results of your query:')
  
  return query_results_df

In [ ]:
query_results_df = chatbot(crumbs_df, stopwords, howmany_v=100, stop_after_multiple_no=6, stop_after_multiple_yes=5)

display(data_table.DataTable(query_results_df.drop(['embeddings'], axis=1), num_rows_per_page=10))

bot: Hello, what are you looking for? 
me:  how much we live
bot: It would help me if you could provide more details. Is 'one room' something related to your query? (yes/no or add some keywords) 
me:  no
bot: I have some results related to 'disposable income'. Is that what you are looking for? (yes/no or add some keywords) 
me:  no
bot: Let's make it more specific. Are you interested in 'years age'? (yes/no or add some keywords) 
me:  yes
bot: Is 'life expectancy' related to what you are looking for? (yes/no or add some keywords) 
me:  yes
bot: Thank you. Here are the most relevant results of your query:


,cross_scores,score,File description,Full path,Link,cluster_id
0,3.010788,0.695110,Life expectancy at a certain age is the mean a...,Life expectancy. Life expectancy at a certain ...,https://ec.europa.eu/eurostat/statistics-expla...,1
1,1.894932,0.527086,"Healthy life years, abbreviated as HLY and als...","Healthy life years (HLY). Healthy life years, ...",https://ec.europa.eu/eurostat/statistics-expla...,1
